# Queries

- "Apple pie"
- "Chicken" in the African category
- "Easy bread" less than 2h
- "Pasta bolognese"
- "Oatmeal"

Prioritize doing 3 of the queries first and running test models, then add more if possible.

# Scoring

Human classification of top 100 results obtained using the standard system (LTR-less). Scoring is done on a numeric scale from 0-5.

## Criteria

The attribution of a given score is a bit subjective but tries to follow the following guidelines:

0. A document that does not match the query.
1. A document that vaguely matches the query, is very incomplete (missing important fields, like instructions) and has no reviews. Or has very negative reviews.
2. A document that partially matches the query, is incomplete and has no reviews. Or a document with negative reviews.
3. A document that matches the query semantically, is reasonably complete (may miss more than two fields) and has at least one positive review.
4. A document that perfectly or almost perfectly matches the query semantically, is complete or missing just one of the fields and has a good number of positive reviews (5 to 20).
5. A document that perfectly matches the query semantically, is complete (the recipe has a full ingredient list, steps and cook time/nutritional information) and has a lot of positive reviews (more than 20).

In [109]:
import urllib.parse as urlp

URL = "http://localhost:8983/solr/recipes/select"
URL += "?rows=100"
URL += "&q.op=AND"
URL += "&q={q}"
URL += "&qf=" + "Name^5 Description Ingredients^2 Keywords^2 Instructions Reviews^0.5 AuthorName^0.2"
URL += "&wt=json"
URL += "&defType=edismax"
URL += "&fl=id,RecipeId,score,[features]"
URL += "&rq={{!ltr model=myModel reRankDocs=100 efi.text={q}}}"
URL += "&fq={fq}"

query = ["apple pie", "chicken", "easy bread", "pasta bolognese", "oatmeal"]
facet = ["", "Category_Facet:African", "", "", ""]
urls = [URL.format(q=query[i], fq=facet[i]) for i in range(len(query))]


In [7]:
import requests
import simplejson
import pandas as pd

for (idx, url) in enumerate(urls):
    response = requests.request("GET", url)
    json = simplejson.loads(response.text)

    for doc in json["response"]["docs"]:
        doc["URL"] = "http://localhost:3000/recipe/{0}".format(doc["RecipeId"]) 
        doc["query"] = query[idx]
        doc["facet"] = facet[idx]
    
    df = pd.DataFrame(json["response"]["docs"])
    df.to_csv("queries/query{0}_results.csv".format(idx+1), index=False)

# Modelling

Solr's LTR implementation supports two different kinds of models: Linear and Tree Based.

There are various algorithms that may be used in order to create these models. We will demonstrate the use of two, one for each type:

- A Linear Model using Support Vector Machines
- A Neural Network model built using RankNet

We will use SciKit Learn's SVM implementation and a RankNet implementation built into Keras.

In [102]:
import pandas as pd
import requests
import simplejson
import glob

result_files = glob.glob("queries/*_results.csv")
scores_files = glob.glob("queries/*_scores.csv")

result_files.sort()
scores_files.sort()

inputs = pd.concat((pd.read_csv(file) for file in result_files), ignore_index=True)
scores = pd.concat((pd.read_csv(file) for file in scores_files), ignore_index=True)

X = []
Y = [entry.score for entry in scores.itertuples()]


In [103]:
def get_features(entry):
    req_url = "http://localhost:8983/solr/recipes/select?rows=100&q.op=AND&q={q}&qf=Name^5%20Description%20Ingredients^2%20Keywords^2%20Instructions%20Reviews^0.5%20AuthorName^0.2&wt=json&defType=edismax&fl=[features]&fq={fq}&rq={rq}"
    facet = entry.facet
    if pd.isna(facet):
        facet = ""
    response = requests.request("GET", req_url.format(q=entry.query, fq=f"RecipeId:{entry.RecipeId} {facet}", rq=f"{{!ltr model=myModel efi.text='{entry.query}'}}"))
    json = simplejson.loads(response.text)
    return [float(feature.split("=")[1]) for feature in json["response"]["docs"][0]["[features]"].split(",")]


for entry in inputs.itertuples():
    X.append(get_features(entry))


In [87]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()

scaler.fit(X)
X = scaler.fit_transform(X)

(train_x,
 test_x,
 train_y,
 test_y) = train_test_split(X, Y, test_size=0.25, random_state=1, stratify=Y)

 

In [104]:
from sklearn import svm, linear_model
from sklearn.metrics import r2_score

linearSVM = svm.LinearSVR()
lienarReg = linear_model.LinearRegression()

linearSVM.fit(train_x, train_y)
lienarReg.fit(train_x, train_y)

pred_svm = linearSVM.predict(test_x)
pred_reg = lienarReg.predict(test_x)

r2_score(test_y, pred_svm)

/home/antonio/.asdf/installs/python/anaconda3-2020.11/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.02679156372444591

In [128]:
with open("train_file.dat", 'w') as file:
    for i in range(len(result_files[:4])):
        in_f = pd.read_csv(result_files[i])
        s_f = pd.read_csv(scores_files[i])
        feats = [" ".join([f"{idx+1}:{f}" for idx, f in enumerate(get_features(entry))]) for entry in in_f.itertuples()]
        r_ids = [entry.RecipeId for entry in in_f.itertuples()]
        scores = [entry.score for entry in s_f.itertuples()]
        file.writelines(f"{s} qid:{i} {f} # {id}\n" for f, s, id in zip(feats, scores, r_ids))



In [118]:
for (i, url) in enumerate(urls):
    response = requests.request("GET", url)
    json = simplejson.loads(response.text)

    scores = pd.read_csv(scores_files[i])

    for doc in json["response"]["docs"]:
        doc["score"] = scores.loc[scores["RecipeId"] == doc["RecipeId"]]["score"].values
        if len(doc["score"]) > 0:
            doc["score"] = doc["score"][0]
        else:
            doc["score"] = ""
        
    df = pd.DataFrame(json["response"]["docs"])
    df.to_csv("queries/query{0}_ltr.csv".format(i+1), index=False)
        

In [119]:
def probability_satisfied(grade):
    return (pow(2, grade) - 1) / 32

def probability_not_satisfied(probs):
    pns = 1
    for p in probs:
        pns = pns * (1 - p)
    return pns

def err(grades):
    err = 0
    probs = []

    for i, grade in enumerate(grades):
        k = i + 1
        ps = probability_satisfied(grade)
        pns = probability_not_satisfied(probs)
        
        err = err + (1 / k) * ps * pns

        probs.append(ps)

    return err

print(err([3, 2, 4]))

0.365997314453125


In [129]:
from statistics import mean

ltr_files = glob.glob("queries/*_ltr.csv")
ltr_files.sort()

og_err = []
ltr_err = []

for (i, ltr_file) in enumerate(ltr_files):
    og_scores = [entry.score for entry in pd.read_csv(scores_files[i]).itertuples()][:10]
    ltr_scores = [entry.score for entry in pd.read_csv(ltr_file).itertuples()][:10]
    og_err.append(err(og_scores))
    ltr_err.append(err(ltr_scores))
    print(f"==== QUERY {i+1} ====")
    print(f"Original: {og_scores}")
    print(f"Original score: {err(og_scores)}")
    print(f"Ltr: {ltr_scores}")
    print(f"Ltr score: {err(ltr_scores)}")
    print()

print(f"Average Original ERR: {mean(og_err)}")
print(f"Average LTR ERR: {mean(ltr_err)}")


==== QUERY 1 ====
Original: [4, 2, 3, 3, 2, 2, 2, 5, 3, 3]
Original score: 0.5889976478421203
Ltr: [3.0, 3.0, 4.0, 3.0, 3.0, 3.0, 5.0, 3.0, 4.0, 3.0]
Ltr score: 0.4573671677939562

==== QUERY 2 ====
Original: [4, 4, 2, 3, 3, 5, 4, 4, 4, 3]
Original score: 0.6505818905264783
Ltr: [4, 5, 4, 1, 0, 2, 2, 0, 0, 0]
Ltr score: 0.728974315870021

==== QUERY 3 ====
Original: [2, 4, 5, 3, 1, 5, 4, 1, 4, 4]
Original score: 0.46439245074792324
Ltr: [3.0, 5.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 5.0, 5.0]
Ltr score: 0.6035056150392544

==== QUERY 4 ====
Original: [3, 2, 2, 3, 2, 2, 2, 3, 3, 2]
Original score: 0.3540176718566535
Ltr: [3, 3, 2, 3, 3, 2, 3, 3, 3, 3]
Ltr score: 0.4015300825310471

==== QUERY 5 ====
Original: [4, 2, 3, 3, 1, 4, 4, 3, 3, 4]
Original score: 0.5895515194293901
Ltr: [4.0, 5.0, 5.0, 3.0, 3.0, 5.0, 5.0, 4.0, 4.0, 4.0]
Ltr score: 0.731533770100546

Average Original ERR: 0.5295082360805131
Average LTR ERR: 0.5845821902669649


# Test Queries

## TQ1: Pumpkin pie

### Regular

- http://localhost:3000/recipe/8451 - 3
- http://localhost:3000/recipe/99678 - 4
- http://localhost:3000/recipe/141358 - 3
- http://localhost:3000/recipe/155999 - 2
- http://localhost:3000/recipe/190375 - 2
- http://localhost:3000/recipe/195917 - 2
- http://localhost:3000/recipe/195979 - 2
- http://localhost:3000/recipe/196600 - 2
- http://localhost:3000/recipe/197120 - 3
- http://localhost:3000/recipe/199458 - 2

### Boosted

- http://localhost:3000/recipe/99678 - 4
- http://localhost:3000/recipe/8451 - 3
- http://localhost:3000/recipe/45262 - 3
- http://localhost:3000/recipe/197120 - 3
- http://localhost:3000/recipe/394476 - 4
- http://localhost:3000/recipe/21047 - 5
- http://localhost:3000/recipe/14186 - 5
- http://localhost:3000/recipe/13888 - 3
- http://localhost:3000/recipe/342359 - 3
- http://localhost:3000/recipe/3558 - 5

In [130]:
og_scores = [3,4,3,2,2,2,2,2,3,2]
ltr_scores = [4,3,3,3,4,5,5,3,3,5]

print(f"==== QUERY pumpkin pie ====")
print(f"Original: {og_scores}")
print(f"Original score: {err(og_scores)}")
print(f"Ltr: {ltr_scores}")
print(f"Ltr score: {err(ltr_scores)}")
print()

==== QUERY pumpkin pie ====
Original: [3, 4, 3, 2, 2, 2, 2, 2, 3, 2]
Original score: 0.4614534571170964
Ltr: [4, 3, 3, 3, 4, 5, 5, 3, 3, 5]
Ltr score: 0.6209241473960555



## TQ2: Vegetable curry

### Regular

- http://localhost:3000/recipe/49821 - 4
- http://localhost:3000/recipe/120637 - 2
- http://localhost:3000/recipe/134115 - 3
- http://localhost:3000/recipe/90061 - 3
- http://localhost:3000/recipe/41642 - 4
- http://localhost:3000/recipe/496890 - 3
- http://localhost:3000/recipe/7801 - 4
- http://localhost:3000/recipe/8316 - 2
- http://localhost:3000/recipe/15279 - 3
- http://localhost:3000/recipe/21098 - 1

### Boosted

- http://localhost:3000/recipe/41642 - 4
- http://localhost:3000/recipe/282585 - 3
- http://localhost:3000/recipe/90061 - 3
- http://localhost:3000/recipe/309611 - 4
- http://localhost:3000/recipe/70361 - 3
- http://localhost:3000/recipe/192353 - 3
- http://localhost:3000/recipe/305712 - 3
- http://localhost:3000/recipe/132070 - 3
- http://localhost:3000/recipe/15279 - 3
- http://localhost:3000/recipe/49821 - 4

In [132]:
og_scores = [4,2,3,3,4,3,4,2,3,1]
ltr_scores = [4,3,3,4,3,3,3,3,3,4]

print(f"==== QUERY vegetable curry ====")
print(f"Original: {og_scores}")
print(f"Original score: {err(og_scores)}")
print(f"Ltr: {ltr_scores}")
print(f"Ltr score: {err(ltr_scores)}")
print()

==== QUERY vegetable curry ====
Original: [4, 2, 3, 3, 4, 3, 4, 2, 3, 1]
Original score: 0.593063969548653
Ltr: [4, 3, 3, 4, 3, 3, 3, 3, 3, 4]
Ltr score: 0.6170006259430124



## TQ3: Mojito

### Regular

- http://localhost:3000/recipe/9452 - 4
- http://localhost:3000/recipe/231513 - 4
- http://localhost:3000/recipe/269483 - 5
- http://localhost:3000/recipe/260760 - 2
- http://localhost:3000/recipe/340514 - 2
- http://localhost:3000/recipe/403427 - 1
- http://localhost:3000/recipe/502066 - 2
- http://localhost:3000/recipe/505828 - 2
- http://localhost:3000/recipe/511090 - 1
- http://localhost:3000/recipe/516940 - 2

### Boosted

- http://localhost:3000/recipe/269483 - 5
- http://localhost:3000/recipe/231513 - 4
- http://localhost:3000/recipe/250822 - 5
- http://localhost:3000/recipe/121715 - 3
- http://localhost:3000/recipe/260323 - 4
- http://localhost:3000/recipe/369597 - 4
- http://localhost:3000/recipe/95096 - 4
- http://localhost:3000/recipe/429288 - 4
- http://localhost:3000/recipe/157973 - 2
- http://localhost:3000/recipe/166675 - 5

In [134]:
og_scores = [4,4,5,2,2,1,2,2,1,2]
ltr_scores = [5,4,5,3,4,4,4,4,2,5]

print(f"==== QUERY mojito ====")
print(f"Original: {og_scores}")
print(f"Original score: {err(og_scores)}")
print(f"Ltr: {ltr_scores}")
print(f"Ltr score: {err(ltr_scores)}")
print()

==== QUERY mojito ====
Original: [4, 4, 5, 2, 2, 1, 2, 2, 1, 2]
Original score: 0.6850325299349714
Ltr: [5, 4, 5, 3, 4, 4, 4, 4, 2, 5]
Ltr score: 0.9815327230797998



## TQ4: Beef stew

### Regular

- http://localhost:3000/recipe/3276 - 3
- http://localhost:3000/recipe/51706 - 2
- http://localhost:3000/recipe/118967 - 2
- http://localhost:3000/recipe/122353 - 3
- http://localhost:3000/recipe/149655 - 4
- http://localhost:3000/recipe/152245 - 1
- http://localhost:3000/recipe/156956 - 2
- http://localhost:3000/recipe/160501 - 2
- http://localhost:3000/recipe/179344 - 2 
- http://localhost:3000/recipe/190865 - 2

### Boosted

- http://localhost:3000/recipe/42730 - 5
- http://localhost:3000/recipe/290875 - 3
- http://localhost:3000/recipe/37242 - 5
- http://localhost:3000/recipe/336537 - 3
- http://localhost:3000/recipe/72839 - 4
- http://localhost:3000/recipe/22341 - 3
- http://localhost:3000/recipe/73799 - 3
- http://localhost:3000/recipe/414009 - 3
- http://localhost:3000/recipe/50334 - 5 
- http://localhost:3000/recipe/11970 - 4

In [137]:
og_scores = [3,2,2,3,4,1,2,2,2,2]
ltr_scores = [5,3,5,3,4,3,3,3,5,4]

print(f"==== QUERY beef stew ====")
print(f"Original: {og_scores}")
print(f"Original score: {err(og_scores)}")
print(f"Ltr: {ltr_scores}")
print(f"Ltr score: {err(ltr_scores)}")
print()

==== QUERY beef stew ====
Original: [3, 2, 2, 3, 4, 1, 2, 2, 2, 2]
Original score: 0.3711691678707563
Ltr: [5, 3, 5, 3, 4, 3, 3, 3, 5, 4]
Ltr score: 0.9801903125328307



## TQ5: Noodles

### Regular

- http://localhost:3000/recipe/76748 - 4
- http://localhost:3000/recipe/337591 - 2
- http://localhost:3000/recipe/342444 - 2
- http://localhost:3000/recipe/106277 - 2
- http://localhost:3000/recipe/260825 - 2
- http://localhost:3000/recipe/355406 - 3
- http://localhost:3000/recipe/9993 - 4
- http://localhost:3000/recipe/10691 - 5
- http://localhost:3000/recipe/14477 - 3
- http://localhost:3000/recipe/15171 - 2

### Boosted

- http://localhost:3000/recipe/61712 - 5
- http://localhost:3000/recipe/177905 - 3
- http://localhost:3000/recipe/136223 - 4
- http://localhost:3000/recipe/182599 - 3
- http://localhost:3000/recipe/167899 - 5
- http://localhost:3000/recipe/9993 - 4
- http://localhost:3000/recipe/211583 - 3
- http://localhost:3000/recipe/76748 - 4
- http://localhost:3000/recipe/70091 - 4 
- http://localhost:3000/recipe/27344 - 4

In [136]:
og_scores = [4,2,2,2,2,3,4,5,3,2]
ltr_scores = [5,3,4,3,5,4,3,4,4,4]

print(f"==== QUERY noodles ====")
print(f"Original: {og_scores}")
print(f"Original score: {err(og_scores)}")
print(f"Ltr: {ltr_scores}")
print(f"Ltr score: {err(ltr_scores)}")
print()

==== QUERY noodles ====
Original: [4, 2, 2, 2, 2, 3, 4, 5, 3, 2]
Original score: 0.5763049125893394
Ltr: [5, 3, 4, 3, 5, 4, 3, 4, 4, 4]
Ltr score: 0.978698259898458

